# PWV01 : Explore hologram data  quality

- author Sylvie Dagoret-Campagne
- creation date 2026-01-22 : version run2026_v01
- last update : 2026-01-23
- affiliation : IJCLab
- Kernel @usdf **w_2026_02*
- Home emac : base (conda)
- laptop : conda_py313

**Goal** : Show Night variations of PWV wrt date and Time. Fit a straight line.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
# must install the mysitcom package by doing at top level "pip install --user -e . "
from mysitcom.auxtel.pwv import scatter_datetime
from mysitcom.auxtel.pwv import strip_datetime
from mysitcom.auxtel.pwv import bar_counts_by_night
from mysitcom.auxtel.pwv import plot_dccd_chi2_vs_time
from mysitcom.auxtel.pwv import plot_dccd_chi2_vs_time_by_filter
from mysitcom.auxtel.pwv import stripplot_target_vs_time
from mysitcom.auxtel.pwv import plot_dccd_chi2_vs_time_by_target_filter
from mysitcom.auxtel.pwv import plot_dccd_chi2_histo_by_target_filter
from mysitcom.auxtel.pwv import plot_dccd_chi2_vs_time_by_target_filter_colorsedtype
from mysitcom.auxtel.pwv import plot_dccd_chi2_histo_by_target_filter_colorsedtype
from mysitcom.auxtel.pwv import summarize_dccd_chi2
from mysitcom.auxtel.pwv import plot_atmparam_vs_time, plot_atmparam_diff_vs_time,plot_atmparam_diff_hist_per_filter

from mysitcom.auxtel.pwv import GetNightMidnightsDict,GetNightBoundariesDict

In [ ]:
import os

In [ ]:
# where are stored the figures
pathfigs = "figs_PWV01"
prefix = "pwv01"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype

import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams["legend.fontsize"] = "xx-large"

import scipy
from scipy.optimize import curve_fit,least_squares

from pprint import pprint

# new color correction model
import pickle
from scipy.interpolate import RegularGridInterpolator

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
# Remove to run faster the notebook
import ipywidgets as widgets
%matplotlib widget

In [ ]:
from PWV00_parameters import *

In [ ]:
DumpConfig()

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
np.__version__

In [ ]:
pd.__version__

### Configuration

In [ ]:
def convertNumToDatestr(num):
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return pd.to_datetime(datestr)

In [ ]:
PWVMIN = 0.
PWVMAX = 20.

In [ ]:
FLAG_WITHCOLLIMATOR = False
DATE_WITHCOLLIMATOR = 20230930
datetime_WITHCOLLIMATOR = convertNumToDatestr(DATE_WITHCOLLIMATOR)
datetime_WITHCOLLIMATOR = pd.to_datetime("2023-09-30 00:00:00.0+0000")
datetime_WITHCOLLIMATOR

## Initialisation

### Read the file
- `atmfilename` is defined in `PWV00_parameters.py` 

In [ ]:
the_suptitle = butlerusercollectiondict[version_run] 

In [ ]:
inputfilename = atmfilename.split("/")[-1]

if "parquet" in inputfilename:
    df_spec = pd.read_parquet(atmfilename)
elif "npy" in inputfilename:
    specdata = np.load(atmfilename,allow_pickle=True)
    df_spec = pd.DataFrame(specdata)
    df_spec["D_CCD [mm]"] = df_spec["D2CCD"]
    df_spec["PWV [mm]"] = df_spec["PWV [mm]_x"] 
    df_spec["PWV [mm]_rum"] = df_spec["PWV [mm]_y"] 
    df_spec["PWV [mm]_err"] = df_spec["PWV [mm]_err_x"] 
    df_spec["PWV [mm]_err_rum"] = df_spec["PWV [mm]_err_y"] 


    cols = [
    "PWV [mm]",
    "PWV [mm]_rum",
    "PWV [mm]_err",
    "PWV [mm]_err_rum",
    ]

    df_spec = df_spec.dropna(subset=cols)
    
else:
    raise "bad path of filename {inputfilename}"
    

In [ ]:
print(" | ".join(df_spec.columns)) 

In [ ]:
#df_spec.dtypes.to_frame('Type de donnée')

In [ ]:
# add time for plotting
#df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"],utc=True)

In [ ]:
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)

In [ ]:
df_spec["seq_num"]  = df_spec["id"] % 100_000

## Select only empty and OG550 filters

In [ ]:
df_spec["FILTER"].unique()

In [ ]:
if FLAG_PWVFILTERS: 
    df_spec = df_spec[df_spec["FILTER"].isin(PWV_FILTER_LIST) ]

### Check Filters

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 8))

strip_datetime(
    df=df_spec,
    x="Time",
    y="FILTER",
    hue="FILTER",
    ax=ax,
    size=9,
)

plt.show()


In [ ]:
bar_counts_by_night(
    df=df_spec,
    night_col="nightObs",
    filter_col="FILTER",
    stacked=True,
    title="Observations per night (stacked)",
)

### Visualize Selection cuts

In [ ]:
df_spec["D2CCD"]

In [ ]:
df_spec["D_CCD [mm]"]

In [ ]:
import matplotlib.pyplot as plt

filters = df_spec["FILTER"].unique()

fig, axs = plt.subplots(len(filters), 2, figsize=(14, 3*len(filters)))

for i, f in enumerate(filters):
    subdf = df_spec[df_spec["FILTER"] == f]  # sélectionne uniquement ce filtre
    
    ax1, ax2 = axs[i] if len(filters) > 1 else axs  # gestion si 1 seul filtre
    
    subdf.hist("D_CCD [mm]", ax=ax1, bins=50,
               range=(DCCDMINFIG, DCCDMAXFIG), facecolor="b")
    ax1.axvline(DCCDMINCUT, ls="-.", c="k")
    ax1.axvline(DCCDMAXCUT, ls="-.", c="k")
    ax1.set_title(f"{f} – D_CCD [mm]_x")

    subdf.hist("CHI2_FIT", ax=ax2, bins=50,range=(0,300) ,facecolor="b")
    #ax2.set_yscale("log")
    ax2.axvline(CHI2CUT, ls="-.", c="k")
    ax2.set_title(f"{f} – CHI2_FIT")
   

  

plt.suptitle(f" Quality selection cut , {tag}")
plt.tight_layout()
plt.show()


In [ ]:
fig, axs = plot_dccd_chi2_vs_time(
    df=df_spec,
    time_col="Time",
    filter_col="FILTER",
    dccd_col="D_CCD [mm]",
    chi2_col="CHI2_FIT",
    dccd_min_fig=DCCDMINFIG,
    dccd_max_fig=DCCDMAXFIG,
    dccd_min_cut=DCCDMINCUT,
    dccd_max_cut=DCCDMAXCUT,
    chi2_cut=CHI2CUT,
    suptitle=tag,
)

plt.tight_layout()
figname =f"{pathfigs}/{prefix}_plot_dccd_chi2_vs_time"+figtype
plt.savefig(figname)
plt.show()

In [ ]:
fig, axs = plot_dccd_chi2_vs_time_by_filter(
    df=df_spec,
    time_col="Time",
    filter_col="FILTER",
    dccd_col="D_CCD [mm]",
    chi2_col="CHI2_FIT",
    dccd_min_fig=DCCDMINFIG,
    dccd_max_fig=DCCDMAXFIG,
    dccd_min_cut=DCCDMINCUT,
    dccd_max_cut=DCCDMAXCUT,
    chi2_cut=CHI2CUT,
    suptitle=tag,
)

plt.tight_layout()
figname =f"{pathfigs}/{prefix}_plot_dccd_chi2_vs_time_by_filter"+figtype
plt.savefig(figname)
plt.show()


### Impact of target on Quality cut

In [ ]:
List_Of_Targets = df_spec["TARGET"].unique()
NTARGETS = len(List_Of_Targets)
print(NTARGETS,List_Of_Targets)

In [ ]:
fig, ax = stripplot_target_vs_time(
    df=df_spec,
    tag=tag,
    figsize=(12,8)
)

plt.tight_layout()
figname =f"{pathfigs}/{prefix}_plot_stripplot_target_vs_time"+figtype
plt.savefig(figname)
plt.show()


### Impact of quality for all targets for empty filter

In [ ]:
fig, axs = plot_dccd_chi2_vs_time_by_target_filter(
    df=df_spec,
    filter_col="FILTER",
    filter_select="empty",
    per_target=False,
    dccd_min_fig=185,   # <- limite inférieure de l'axe
    dccd_max_fig=190,   # <- limite supérieure de l'axe
    dccd_min_cut=DCCDMINCUT,
    dccd_max_cut=DCCDMAXCUT,
    chi2_min_fig=1.,   
    chi2_max_fig=6000.,   
    chi2_cut=CHI2CUT,
    suptitle="empty – all targets",
    tag = tag
)

figname =f"{pathfigs}/{prefix}_plot_dccd_chi2_vs_time_all-targets_filter-empty"+figtype
plt.savefig(figname)
plt.show()

### Impact of quality for all targets for OG550 filter

In [ ]:
fig, axs = plot_dccd_chi2_vs_time_by_target_filter(
    df=df_spec,
    filter_col="FILTER",
    filter_select="OG550_65mm_1",
    per_target=False,
    dccd_min_fig=185,   # <- limite inférieure de l'axe
    dccd_max_fig=190,   # <- limite supérieure de l'axe
    dccd_min_cut=DCCDMINCUT,
    dccd_max_cut=DCCDMAXCUT,
    chi2_min_fig=1.,   
    chi2_max_fig=6000.,   
    chi2_cut=CHI2CUT,
    suptitle="OG550 – all targets",
    tag = tag
)

plt.tight_layout()
figname =f"{pathfigs}/{prefix}_plot_dccd_chi2_vs_time_all-targets_filter-og550"+figtype
plt.savefig(figname)
plt.show()


## Histograms on quality cuts

In [ ]:
fig, axs = plot_dccd_chi2_histo_by_target_filter(
    df=df_spec,
    filter_select="empty",
    per_target=False,
    dccd_min_fig=186,
    dccd_max_fig=189,
    dccd_min_cut=DCCDMINCUT,
    dccd_max_cut=DCCDMAXCUT,
    chi2_min_fig=1,
    chi2_max_fig=6000,
    chi2_cut=CHI2CUT,
    suptitle="empty – histograms quality cuts",
)

In [ ]:
fig, axs = plot_dccd_chi2_histo_by_target_filter(
    df=df_spec,
    filter_select="OG550_65mm_1",
    per_target=False,
    dccd_min_fig=186,
    dccd_max_fig=189,
    dccd_min_cut=DCCDMINCUT,
    dccd_max_cut=DCCDMAXCUT,
    chi2_min_fig=1,
    chi2_max_fig=6000,
    chi2_cut=CHI2CUT,
    suptitle="OG550 – histograms quality cuts",
)

## Special study on Star Color (Spectral type)
- Load magnitudes from external file
- the magnitudes have been computed in another notebooks in ../2025-10-29-TOOLS



In [ ]:
targets_mag_files = "../2025-10-29-TOOLS/data/targets_magnitudes.csv"
df_targets_mag = pd.read_csv(targets_mag_files,index_col=0)      
df_targets_mag = df_targets_mag.sort_values(by="y")

### palette with SED type

In [ ]:
df_col = df_targets_mag.copy()
df_col = df_col.sort_values(by="B_V")

SpT = df_col["Sp_T"].values
unique_types = list(dict.fromkeys(SpT))  # garde l'ordre d'apparition
N_types = len(unique_types)


# Associe chaque type spectral à un entier
type_to_idx = {t: i for i, t in enumerate(unique_types)}
idx = np.array([type_to_idx[t] for t in SpT])

# Crée la colormap
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=-0.5, vmax=N_types - 0.5)

# Colorbar horizontale
fig, ax = plt.subplots(figsize=(14, 0.4), layout="constrained")
cbar = fig.colorbar(
    mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
    cax=ax,
    orientation="horizontal",
    ticks=range(N_types),
)
cbar.ax.set_xticklabels(unique_types, rotation=45, ha="right")
cbar.set_label("Spectral Type", fontsize=12,labelpad=10)

plt.show()

In [ ]:
df_targets_mag.head() 

### Target ordered by colors in the input file

In [ ]:
sorted_targets =  list(df_targets_mag.index) 
print(sorted_targets)

### Target ordered by colors in our data  file

In [ ]:
# order pf selected target by magnitude Y
order_selected_targets = [t for t in sorted_targets if t in df_spec["TARGET"].unique()]
print(order_selected_targets)

### build a palette for colors

In [ ]:
# --- Palette personnalisée cohérente avec la colormap jet ---
target_to_color = {}
for target in order_selected_targets:
    sp_type = df_targets_mag.loc[target, "Sp_T"]
    rgba = cmap(norm(type_to_idx[sp_type]))

    # Convert to pure Python floats
    target_to_color[target] = tuple(float(c) for c in rgba)

In [ ]:
fig, axs = plot_dccd_chi2_vs_time_by_target_filter_colorsedtype(
    df=df_spec,
    filter_col="FILTER",
    filter_select="empty",
    per_target=False,
    dccd_min_fig=185,   # <- limite inférieure de l'axe
    dccd_max_fig=190,   # <- limite supérieure de l'axe
    dccd_min_cut=DCCDMINCUT,
    dccd_max_cut=DCCDMAXCUT,
    chi2_min_fig=1.,   
    chi2_max_fig=6000.,   
    chi2_cut=CHI2CUT,
    suptitle="empty – all targets (color SED-type)",
    tag = tag,
    target_palette=target_to_color
)

figname =f"{pathfigs}/{prefix}_plot_dccd_chi2_vs_time_all-targets_filter-empty_colorsedtype"+figtype
plt.savefig(figname)
plt.show()

In [ ]:
fig, axs = plot_dccd_chi2_histo_by_target_filter_colorsedtype(
    df=df_spec,
    filter_select="empty",
    per_target=False,
    dccd_min_fig=186,
    dccd_max_fig=189,
    dccd_min_cut=DCCDMINCUT,
    dccd_max_cut=DCCDMAXCUT,
    chi2_min_fig=1,
    chi2_max_fig=6000,
    chi2_cut=CHI2CUT,
    target_palette=target_to_color,
    suptitle="empty – histograms",
    tag = tag,
)
figname =f"{pathfigs}/{prefix}_plot_dccd_chi2_hosto_all-targets_filter-empty_colorsedtype"+figtype
plt.savefig(figname)
plt.show()

In [ ]:
fig, axs = plot_dccd_chi2_vs_time_by_target_filter_colorsedtype(
    df=df_spec,
    filter_col="FILTER",
    filter_select="OG550_65mm_1",
    per_target=False,
    dccd_min_fig=185,   # <- limite inférieure de l'axe
    dccd_max_fig=190,   # <- limite supérieure de l'axe
    dccd_min_cut=DCCDMINCUT,
    dccd_max_cut=DCCDMAXCUT,
    chi2_min_fig=1.,   
    chi2_max_fig=6000.,   
    chi2_cut=CHI2CUT,
    suptitle="OG550 – all targets (color SED-type)",
    tag = tag,
    target_palette=target_to_color
)

figname =f"{pathfigs}/{prefix}_plot_dccd_chi2_vs_time_all-targets_filter-og550_colorsedtype"+figtype
plt.savefig(figname)
plt.show()

In [ ]:
fig, axs = plot_dccd_chi2_histo_by_target_filter_colorsedtype(
    df=df_spec,
    filter_select="OG550_65mm_1",
    per_target=False,

    dccd_min_fig=186,
    dccd_max_fig=189,
    dccd_min_cut=DCCDMINCUT,
    dccd_max_cut=DCCDMAXCUT,
    chi2_min_fig=1,
    chi2_max_fig=6000,
    chi2_cut=CHI2CUT,
    target_palette=target_to_color,
    suptitle="OG550 – histograms",
    tag = tag,
)
figname =f"{pathfigs}/{prefix}_plot_dccd_chi2_hosto_all-targets_filter-og550_colorsedtype"+figtype
plt.savefig(figname)
plt.show()

# Plot PWV vs time before application cuts

In [ ]:
DT = pd.Timedelta(minutes=7*24*60)
TMIN  = df_spec["Time"].min()-DT
TMAX  = df_spec["Time"].max()+DT

In [ ]:
# get night boundaries
dn = GetNightBoundariesDict(df_spec)
# get midnights
dnidnights = GetNightMidnightsDict(df_spec)

In [ ]:
fig,ax = plot_atmparam_vs_time(
    df_spec,
    time_col= "Time",
    filter_col = "FILTER",
    param_col = "PWV [mm]",
    param_err_col = "PWV [mm]_err",
    title_param = "PWV vs time (spectrogram no cut qual. cut)",
    
    # seuils / bornes
    param_min_fig=PWVMIN,
    param_max_fig=PWVMAX,
    param_min_cut=None,
    param_max_cut=None,
 
    # titres
    suptitle= the_suptitle,

    # axes externes
    axs=None,
    figsize=(18, 6),
)

for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.05,lw=0.5)

for key, midn in dnidnights.items():
    ax.axvline( midn ,color="purple",ls=":",alpha=0.5,lw=0.5)

In [ ]:
fig,ax = plot_atmparam_vs_time(
    df_spec,
    time_col= "Time",
    filter_col = "FILTER",
    param_col = "PWV [mm]_rum",
    param_err_col = "PWV [mm]_err_rum",
    title_param = "PWV vs time (spectrum no cut qual. cut)",
    
    # seuils / bornes
    param_min_fig=PWVMIN,
    param_max_fig=PWVMAX,
    param_min_cut=None,
    param_max_cut=None,
 
    # titres
    suptitle= the_suptitle,

    # axes externes
    axs=None,
    figsize=(18, 6),
)

for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.05,lw=0.5)

for key, midn in dnidnights.items():
    ax.axvline( midn ,color="purple",ls=":",alpha=0.5,lw=0.5)

In [ ]:
fig,ax = plot_atmparam_diff_vs_time(
    df_spec,
    time_col= "Time",
    filter_col = "FILTER",
    param1_col = "PWV [mm]",
    param2_col = "PWV [mm]_rum",
    param1_err_col = "PWV [mm]_err",
    param2_err_col = "PWV [mm]_err_rum",
    title_param = "$\Delta$ PWV vs time (spectrogram - spectrum no cut qual. cut)",
    
    # seuils / bornes
    param_min_fig=-PWVMAX/10.,
    param_max_fig=PWVMAX/10.,
    param_min_cut=None,
    param_max_cut=None,
 
    # titres
    suptitle= the_suptitle,

    # axes externes
    axs=None,
    figsize=(18, 6),
)

for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.05,lw=0.5)

for key, midn in dnidnights.items():
    ax.axvline( midn ,color="purple",ls=":",alpha=0.5,lw=0.5)

In [ ]:
fig,ax = plot_atmparam_diff_hist_per_filter(
    df_spec,
    filter_col="FILTER",
    param1_col = "PWV [mm]",
    param2_col= "PWV [mm]_rum",
    paramdiff_range = (-PWVMAX/10.,PWVMAX/10.),

    # histogram control
    bins=100,
    density=True,
    hist_alpha=0.4,

    # x-axis limits
    param_min_fig=-PWVMAX/10,
    param_max_fig=PWVMAX/10.,

    title_param="$\Delta$ PWV vs time (spectrogram - spectrum no cut qual. cut)"
)
plt.show()

# Apply Selection cuts  

In [ ]:
df_spec_sel =  getSelectionCutforPWV(df_spec,
                          pwv_ram_col = "PWV [mm]",
                          pwv_rum_col = "PWV [mm]_rum",
                          chi2_col = "CHI2_FIT",
                          d2ccd_col = "D2CCD",
                          exptime_col = 'EXPTIME')

# Check impact of selection cuts on PWV quality

In [ ]:
fig,ax = plot_atmparam_diff_hist_per_filter(
    df_spec_sel,
    filter_col="FILTER",
    param1_col = "PWV [mm]",
    param2_col= "PWV [mm]_rum",
    paramdiff_range = (-PWVMAX/10.,PWVMAX/10.),

    # histogram control
    bins=100,
    density=True,
    hist_alpha=0.4,

    # x-axis limits
    param_min_fig=-PWVMAX/10,
    param_max_fig=PWVMAX/10.,

    title_param="$\Delta$ PWV vs time (spectrogram - spectrum no cut qual. cut)"
    )
plt.show()

In [ ]:
fig,ax = plot_atmparam_diff_vs_time(
    df_spec_sel,
    time_col= "Time",
    filter_col = "FILTER",
    param1_col = "PWV [mm]",
    param2_col = "PWV [mm]_rum",
    param1_err_col = "PWV [mm]_err",
    param2_err_col = "PWV [mm]_err_rum",
    title_param = "$\Delta$ PWV vs time (spectrogram - spectrum after cut qual. cut)",
    
    # seuils / bornes
    param_min_fig=-PWVMAX/10.,
    param_max_fig=PWVMAX/10.,
    param_min_cut=None,
    param_max_cut=None,
 
    # titres
    suptitle= the_suptitle,

    # axes externes
    axs=None,
    figsize=(18, 6),
)

for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.05,lw=0.5)

for key, midn in dnidnights.items():
    ax.axvline( midn ,color="purple",ls=":",alpha=0.5,lw=0.5)